In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy scipy pandas matplotlib numpy_ringbuffer sklearn

import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('..')

In [ ]:
# currencies = ['usd', 'btc', 'eth', 'ltc', 'xrp', 'eos']
# pairs = [c + '_usd' for c in currencies if c != 'usd']
# volume_keys = [c + '_tx_volume' for c in currencies if c != 'usd']

# def prep_data(file):
#     data = pickle.load(open(file, 'rb'))
#     dates = [x['date'] for x in data]
#     prices = [{k:v for k,v in x.items() if k in pairs} for x in data]
#     volumes = [{(k.partition('_')[0] + '_usd'):v for k,v in x.items() if k in volume_keys} for x in data]
#     return {
#         'prices': pd.DataFrame(prices, index = dates),
#         'volumes': pd.DataFrame(volumes, index = dates)
#     }

# def reduce_data(data, resampling):
#     '''Averages prices, sums volumes'''
#     prices = data['prices'].resample(resampling).first().fillna(method='ffill')
#     volumes = data['volumes'].resample(resampling).sum().fillna(method='ffill')
#     return { 'prices': prices, 'volumes': volumes }

# def tail_data(data, n):
#     '''get the last n points of the given data'''
#     prices = data['prices'].tail(n)
#     volumes = data['volumes'].tail(n)
#     return { 'prices': prices, 'volumes': volumes }

# def viz_data(data):
#     '''Only plots prices for now'''
#     plt.plot(data['prices'] / data['prices'].mean() - 1)
#     plt.show()

# def find_gaps(data, freq):
#     idx_ref = pd.date_range(start=data.index[0], end=data.index[-1],freq=freq)
#     gaps = idx_ref[~idx_ref.isin(data.index)]
#     return gaps

# data = prep_data('data/data.p')
# data_min = reduce_data(prep_data('data/data-minute.p'), '1Min')
# data_5min = reduce_data(data_min, '5Min')
# data_15min = reduce_data(data_min, '15Min')
# viz_data(data_min)

In [ ]:
data_min = pd.read_hdf('data/1min.h5')
data_15min = data_min.resample('15Min').first()
data_1h = data_min.resample('1h').first()

In [ ]:
from trader.util.stats import Ema, HoltEma
price_data = data_15min.xs('price', axis=1, level=1)
h = HoltEma(64, 512)
smooth = price_data.apply(h.step, axis=1).rename(columns=lambda x: str(x) + '_smooth')
together = pd.concat([price_data, smooth], axis=1)
(together/together.mean()).plot(figsize=(16,10))

In [ ]:
from copy import deepcopy
h = HoltEma(3, 3)
def htrend(d):
    h.step(d)
    return h.trend
price_data_ = price_data[-300:].filter(regex="-LTC-")
trend = price_data_.apply(htrend, axis=1).rename(columns=lambda x: str(x) + '_trend')
e = Ema(9)
ema_trend = price_data_.diff()[1:].apply(e.step, axis=1).rename(columns=lambda x: str(x) + '_ema_trend')
h2 = HoltEma(9, 9)
hema_trend = price_data_.diff()[1:].apply(h2.step, axis=1).rename(columns=lambda x: str(x) + '_hema_trend')
(price_data_ / price_data_.mean()).plot(figsize=(16,10))
together = pd.concat([trend, ema_trend, hema_trend], axis=1)
((together - together.mean()) / together.std()).plot(figsize=(16,10))

In [ ]:
from trader.util.types import TradingPair, ExchangePair
from trader.util.constants import LTC, USDT, BINANCE
price_data_ = price_data[-100:].filter(regex="-LTC-")
diffs = price_data_.diff()[1:]
diffs /= diffs.std()
x = diffs.abs().sort_values(by=ExchangePair(BINANCE, TradingPair(LTC, USDT))).reset_index(drop=True)
diffs.plot()
x.plot()
price_data_.plot()

In [ ]:
# def plot_matrix(df,size=10):
#     '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

#     Input:
#         df: pandas DataFrame
#         size: vertical and horizontal size of the plot'''

#     fig, ax = plt.subplots(figsize=(size, size))
#     ax.matshow(df)
#     plt.xticks(range(len(df.columns)), df.columns);
#     plt.yticks(range(len(df.index)), df.index);
#     # Loop over data dimensions and create text annotations.
#     for i in range(len(df.index)):
#         for j in range(len(df.columns)):
#             ax.text(j, i, '{:0.2f}'.format(df.iloc[i, j]), ha="center", va="center", color="w")

# plot_matrix(data_15min['prices'].corr())

In [ ]:
# def cross_correlate_(x, y):
#     return np.argmax(np.correlate(x, y, mode='full')) - len(x) + 1

# def cross_correlate(df):
#     '''Compute cross-correlation matrix for the given dataframe.'''
#     ccs = pd.DataFrame(index=df.columns, columns=df.columns)
#     for i in df.columns:
#         for j in df.columns:
#             if i == j:
#                 ccs.loc[i,j] = 0
#                 continue
#             if np.isnan(ccs.loc[i,j]):
#                 ccs.loc[i,j] = cross_correlate_(df[i], df[j])
#                 ccs.loc[j,i] = -ccs.loc[i,j]
#     return ccs

# print(cross_correlate(pd.DataFrame([[1,2],[2,1],[1,2],[2,1],[1,2]])))
# print(cross_correlate(pd.DataFrame([[1,1],[2,2],[3,3],[4,4],[5,5]])))
    
# print(cross_correlate(data_min['prices']))

In [ ]:
import pandas as pd

from trader.util.constants import BCH, BTC, EOS, ETH, LTC, NEO, XRP
from trader.util.stats import Ema

# TODO: fetch this dynamically
CIRCULATING_SUPPLY = pd.Series(
    {BTC: 18e6, ETH: 106e6, XRP: 42e9, BCH: 18e6, EOS: 913e6, LTC: 62e6, NEO: 65e6}
)

# TODO: actually convert quotes, not just volume
# think about how to do this. what if a quote_usd pair does not exist on a :particular exchange?
def convert_quotes_to_usd(frame):
    frame = frame.copy()
    # .values call necessary because assigning to indexslices is buggy
    # see https://github.com/pandas-dev/pandas/issues/10440
    frame.loc[pd.IndexSlice[:, "volume"]] = (frame.xs("volume", level=1) * frame.xs("price", level=1)).values
    return frame


def aggregate_currency_quotes(moving_volumes, frame):
    """
    Aggregate quotes for the same base currency across all exchanges and pairs. Quotes should
    already be in USD.

    Returns aggregate quotes.

    Note: this may cause unwanted fluctuation in the aggregate price if quotes disagree - and
    trading volume moves back and forth between them. To combat this we weight quotes by a slow
    moving average of trading volume, but is there a better solution?
    """
    currencies = {pair.base for pair in frame.index.unique(level=0)}
    index = pd.MultiIndex.from_product([currencies, frame.index.unique(level=1)])
    aggregates = pd.Series(index=index)
    for c in currencies:
        components = frame.filter(regex="-" + str(c) + "-.*", axis=0)
        moving_volumes_c = moving_volumes.filter(regex="-" + str(c) + "-.*") + 1e-10
        volume = components.xs("volume", level=1).sum()
        price = components.xs("price", level=1) @ moving_volumes_c / moving_volumes_c.sum()
        aggregates.loc[pd.IndexSlice[c, :]] = (price, volume)
    
    return aggregates


def compute_baskets(basket_specs, aggregates):
    """
    Compute a cap-weighted basket of currencies. Input should have aggregate quotes.
    """
    index = pd.MultiIndex.from_product([basket_specs, aggregates.index.unique(level=1)])
    baskets = pd.Series(index=index)
    for name, currencies in basket_specs.items():
        components = aggregates.loc[pd.IndexSlice[currencies, :]]
        # scale down to avoid numerical instability
        price = components.xs("price", level=1) @ CIRCULATING_SUPPLY[currencies] / 1e9
        volume = components.xs("volume", level=1).sum()
        baskets.loc[pd.IndexSlice[name, :]] = (price, volume)
    return baskets

class SignalAggregator:
    """
    Adds cap-weighted baskets to frame.
    TODO: also convert non-USD quotes to USD, add aggregated currency prices
    """

    def __init__(self, volume_half_life, baskets):
        self.__moving_volumes = Ema(volume_half_life)
        self.__baskets = baskets

    def step(self, frame):
        frame = convert_quotes_to_usd(frame)
        moving_volumes = self.__moving_volumes.step(frame.xs("volume", level=1))
        aggregates = aggregate_currency_quotes(moving_volumes, frame)
        baskets = compute_baskets(self.__baskets, aggregates)
        signals = pd.concat([aggregates, baskets]).astype(np.float64)
        return signals, frame
        
# TODO: vectorize this
def prep_warmup_data(data, signal_aggregator, warmup_len):
    warmup = data.iloc[:warmup_len]
    data = data.iloc[warmup_len:]
    signal_frames, data_frames = zip(*[signal_aggregator.step(frame) for _, frame in warmup.iterrows()])
    warmup_signals = pd.DataFrame(signal_frames, index=warmup.index)
    warmup_data = pd.DataFrame(data_frames, index=warmup.index)
    return (warmup_data, warmup_signals), data
    
signal_aggregator_15min = SignalAggregator(500, { "total_market": [BTC, ETH, XRP, LTC, EOS, NEO] })
(warmup_data_15min, warmup_signals_15min), data_15min_ = prep_warmup_data(data_15min, signal_aggregator_15min, 500)
print(warmup_signals_15min.tail())

In [ ]:
import numpy as np
from trader.util.stats import HoltEma, Emse, TrendEstimator
from trader.util.maths import sigmoid

class ExecutionStrategy:
    def __init__(self, size, trend_hl, accel_hl, fair_trend_hl, fair_accel_hl, warmup_data):
        self.size = size
        # Does it make sense to replace with something else? (like % of takes that are buys)
        
        warmup_prices = warmup_data.xs("price", axis=1, level=1)
        most_recent_prices = warmup_prices.iloc[-1]
        self.trend_estimator = TrendEstimator(HoltEma(trend_hl, accel_hl), most_recent_prices)
        self.fair_trend_estimator = TrendEstimator(HoltEma(fair_trend_hl, fair_accel_hl), most_recent_prices)
        for _, prices in warmup_prices.iloc[-4 * fair_accel_hl:].iterrows():
            self.trend_estimator.step(prices)
            self.fair_trend_estimator.step(prices) # should this be different?

    def tick(self, positions, prices, fairs, fees):
        """Takes fair as Gaussian, positions in base currency.
        Returns orders in base currency (negative size indicates sell).

        Since our fair estimate may have skewed uncertainty, it may be the case that
        a price change for one trading pair causes us to desire positions in other
        pairs. Therefore get_orders needs to consider the entire set of fairs and
        bid/asks at once.

        TODO: generalize to multiple quotes
        TODO: use books instead of just the best bid/ask price
        """
        # bookkeepping
        trend = self.trend_estimator.step(prices)
        fair_trend = self.fair_trend_estimator.step(fairs.mean)
        
        pct_edge = fairs.mean / prices - 1
        net_pct_edge = np.sign(pct_edge) * np.maximum(0, np.abs(pct_edge) - 2 * fees)
        z_edge = (fairs.mean - prices) / fairs.stddev
        target_position_values = np.sign(pct_edge) * np.sqrt(z_edge * net_pct_edge * 100) * self.size
        pair_positions = positions[[(ep.exchange_id, ep.base) for ep in prices.index]].set_axis(
            prices.index, inplace=False
        )
        proposed_orders = target_position_values / fairs.mean - pair_positions
        profitable = np.sign(proposed_orders) * pct_edge > 2 * fees
        trending_correctly = (trend * np.sign(pct_edge) > 0) & (fair_trend * np.sign(pct_edge) > 0)
        profitable_orders = proposed_orders * profitable * trending_correctly

        unprofitable_position = np.sign(pair_positions) * pct_edge < 0
        position_closing_orders = -pair_positions * (profitable_orders == 0) * unprofitable_position * trending_correctly

        return profitable_orders + position_closing_orders


In [ ]:
##### import numpy as np
import pandas as pd
from trader.util import Gaussian
from abc import ABC, abstractmethod
from trader.util.stats import HoltEma

# Note: Assumes all orders fill at last trade price. Attempting to simulate market-making would
# require combing through book and trade data, which is too much work for us to do at the moment.


def execute_orders(fees, prices, balances, orders):
    for (ep, size) in orders.items():
        value = size * prices[ep]
        balances[(ep.exchange_id, ep.quote)] -= value
        balances[(ep.exchange_id, ep.quote)] -= abs(value) * fees
        balances[(ep.exchange_id, ep.base)] += size


def run(
    strategy,
    execution_strategy,
    data,
    aggregator = SignalAggregator(1,{}),
    fees=0
):
    eps = data.columns.unique(0)
    balances = pd.Series(0.0, index=[(ep.exchange_id, ep.base) for ep in eps])
    quote_balances = pd.Series(0.0, index={(ep.exchange_id, ep.quote) for ep in eps})
    balances = balances.append(quote_balances)
    balance_history = []
    fair_history = []
    print("running")
    for _, frame in data.iterrows():
        signals, frame = aggregator.step(frame)
#         print(signals)
        fairs = strategy.tick(frame, signals)
        fairs &= Gaussian(frame.xs('price',level=1), [1e100 for _ in eps])
        
        orders = execution_strategy.tick(balances, frame.xs('price',level=1), fairs, fees)
        execute_orders(fees, frame.xs('price',level=1), balances, orders)

        fair_history.append(fairs)
        balance_history.append(balances.copy())
    return {
        'data': data,
        'fairs': pd.DataFrame(fair_history, index=data.index),
        'balances': pd.DataFrame(balance_history, index=data.index)
    }


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# number of ticks to sum for price movements in risk calculation
RISK_WINDOW = 10


def principal_market_movements(prices):
    """Returns principal vectors for 1-stddev market movements, plus explained variance ratios"""
    # Fit PCA to scaled (mean 0, variance 1) matrix of single-tick price differences
    pca = PCA(n_components=0.97)
    scaler = StandardScaler()
    price_deltas = prices.diff().iloc[1:].rolling(RISK_WINDOW).sum().iloc[RISK_WINDOW:]
    price_deltas_scaled = scaler.fit_transform(price_deltas)
    pca.fit(price_deltas_scaled)
    pcs = pd.DataFrame(scaler.inverse_transform(pca.components_), columns=price_deltas.columns)
    return (pcs, pca.explained_variance_ratio_)


def max_abs_drawdown(pnls):
    """Maximum peak-to-trough distance before a new peak is attained. The usual metric, expressed
    as a fraction of peak value, does not make sense in the infinite-leverage context."""
    max_drawdown = 0
    peak = -np.inf
    for pnl in pnls:
        if pnl > peak:
            peak = pnl
        drawdown = peak - pnl
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown


def analyze(results, plot=True):
    """Analyzes P/L and various risk metrics for the given run results.
    Plots balances (with P/L) and market risk over time.

    Note: RoRs are per-tick. They are NOT comparable across time scales."""
    # Balance values
    price_data = results["data"].xs("price", axis=1, level=1)
    quote_currency = (price_data.columns[0].exchange_id, price_data.columns[0].quote)
    prices_ = price_data.rename(columns=lambda pair: (pair.exchange_id, pair.base))
    prices_[quote_currency] = 1
    balance_values = results["balances"] * prices_

    pnls = balance_values.sum(axis=1)
    pnl = pnls.iloc[-1]

    # Market risk
    (pmms, pmm_weights) = principal_market_movements(price_data)
    balances_ = results["balances"][
        [(pair.exchange_id, pair.base) for pair in price_data.columns]
    ].set_axis(price_data.columns, axis=1, inplace=False)
    component_risks = np.abs(balances_ @ pmms.T)
    risks = component_risks @ pmm_weights

    total_positions = np.abs(balance_values.drop(columns=[quote_currency]).values).sum()
    max_drawdown = max_abs_drawdown(pnls)

    if plot:
        fig, axs = plt.subplots(figsize=(16, 4))
        balance_values.plot(ax=axs)
        pd.DataFrame(pnls, columns=["P/L"]).plot(ax=axs)
        axs.axhline(0, color="grey")
        plt.show()
        print(f"Return on maximum market risk: {pnl / (risks.values.max() + 1e-10)}")
        print(f"Return on maximum drawdown:    {pnl / (max_drawdown + 1e-10)}")
        print(f"Return on total market risk:   {pnl / (risks.values.sum() + 1e-10)}")
        print(f"Return on total positions:     {pnl / (total_positions + 1e-10)}")
        print(f"Sharpe ratio:                  {pnl / (pnls.std() + 1e-10)}")
        print(f"Final P/L:                     {pnl}")
        print(f"Maximum market risk:           {risks.values.max()}")
        print(f"Maximum drawdown:              {max_drawdown}")
        print(f"Final balances:")
        print(results["balances"].iloc[-1])

    return pnl / (risks.values.max() + 1e-10)


In [ ]:
from trader.strategy import Strategy
from trader.util import Gaussian

class DummyStrategy(Strategy):
    """A strategy that always returns a null prediction."""

    def tick(self, frame, signals):
        return Gaussian(pd.Series([]), [])


execution_strategy = ExecutionStrategy(1000, 500, 1, 3, 2, 6, warmup_data_15min)
analyze(run(DummyStrategy(), execution_strategy, data_min.tail(50)))

In [ ]:
signal_aggregator_min = SignalAggregator(7500, { "total_market": [BTC, ETH, XRP, LTC, EOS, NEO] })
(warmup_data_min, warmup_signals_min), data_min_ = prep_warmup_data(data_min, signal_aggregator_min, 7500)
print(warmup_signals_min.tail())

In [ ]:
### from trader.strategy.base import Strategy
from trader.util.stats import Ema, Emse, HoltEma
from trader.util import Gaussian
from copy import deepcopy

from trader.util.constants import BCH, BTC, EOS, ETH, LTC, NEO, XRP
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
from statsmodels.tsa.stattools import coint
import warnings

def bhattacharyya_multi(a, b):
    """
    Vectorized calculation of Bhattacharyya distance for 1-d Gaussians.
    Formula from https://en.wikipedia.org/wiki/Bhattacharyya_distance
    """
    return (1/4) * (
        np.log((1/4) * (a.variance / b.variance + b.variance / a.variance + 2))
        + (a.mean - b.mean) ** 2 / (a.variance + b.variance)
    )

def intersect_with_disagreement(gaussians):
    intersection = Gaussian.intersect(gaussians)
    disagreement = pd.DataFrame([bhattacharyya_multi(intersection, g) for g in gaussians])
    return Gaussian(intersection.mean, intersection.variance * (1 + disagreement.pow(2).sum()))

def remove_trend(df):
    """Remove linear trend from the input data by subtracting the OLS."""
    A = np.vstack([df.index.values, np.ones(len(df.index))]).T
    Q = np.linalg.lstsq(A, df, rcond=None)[0]
    trend = A @ Q
    return df - trend

class KalmanFilter(Strategy):
    '''
    Models fairs based on correlated movements between pairs. Weights predictions by volume and
    likelihood of cointegration.
    '''

    def __init__(
        self,
        window_size,
        movement_hl,
        trend_hl,
        mse_hl,
        cointegration_period,
        maxlag,
        warmup_signals,
        warmup_data,
    ):
        self.window_size = window_size
        self.moving_prices = HoltEma(movement_hl, trend_hl, mse_hl)
        self.moving_err_from_prev_fair = Emse(mse_hl)
        self.cointegration_period = cointegration_period
        self.maxlag = maxlag
        self.sample_counter = 0
        self.r = None
        self.r2 = None
        # TODO: do some checks for length/pairs of warmup signals/outputs
        prices = pd.concat([warmup_signals.xs("price", axis=1, level=1), warmup_data.xs("price", axis=1, level=1)], axis=1, sort=False)
        volumes = pd.concat([warmup_signals.xs("volume", axis=1, level=1), warmup_data.xs("volume", axis=1, level=1)], axis=1, sort=False)
        self.price_history = RingBuffer(self.window_size, dtype=(np.float64, len(prices.columns)))
        self.price_history.extend(prices.values)

        for _, p in prices.iloc[-trend_hl * 4 :].iterrows():
            self.moving_prices.step(p)

        self.moving_volumes = Ema(mse_hl, volumes.mean())

        self.moving_variances = TrendEstimator(Emse(window_size / 2, (prices.diff()[1:] ** 2).mean()), prices.iloc[-1])

        self.prev_fair = Gaussian(self.moving_prices.value, [1e100 for _ in prices.columns])

        self.coint_f = pd.DataFrame(1, index=warmup_signals.columns.unique(0), columns=prices.columns)

    # the fair combination step assumes that all signals are i.i.d. They are not (and obviously not in the case
    # of funds). Is this a problem?
    def tick(self, frame, signals):
        prices = pd.concat([signals.xs("price", level=1), frame.xs("price", level=1)], sort=False)
        volumes = pd.concat([signals.xs("volume", level=1), frame.xs("volume", level=1)], sort=False)
        input_names = prices.index
        signal_names = signals.index.unique(0)
        if self.price_history is None:
            # TODO: will be buggy
            self.price_history = RingBuffer(
                self.window_size, dtype=(np.float64, len(input_names)))
            # duplicate the first value so we can compute the most recent movement without having to check
            self.price_history.append(prices)

        self.price_history.append(prices)

        price_history = pd.DataFrame(self.price_history, columns=input_names)

        moving_prices = self.moving_prices.step(prices)
        moving_volumes = self.moving_volumes.step(volumes)
        stddev = np.sqrt(self.moving_variances.step(prices))

        if len(self.price_history) < self.window_size or not self.moving_prices.ready:
            return Gaussian(pd.Series([]),[])

        if self.prev_fair is None:
            self.prev_fair = Gaussian(moving_prices, [1e100 for _ in input_names])
            
        if self.coint_f is None:
            self.coint_f = pd.DataFrame(1, index=signal_names, columns=input_names)
            
        # The moving average is already trend-compenstated, so we remove trend from the data.
        # price_history = remove_trend(price_history)

        # calculate p values for pair cointegration
        if self.sample_counter == 0:
            for i in signal_names:
                for j in input_names:
                    # ignore collinearity warning
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore")
                        p = coint(price_history[i], price_history[j], trend='ct', maxlag=self.maxlag, autolag=None)[1]
                        self.coint_f.loc[i,j] = 1 + p * p * p * 15625 # .04 -> 2, .05 -> ~3, .1 -> 15.625
            self.r = price_history.corr().loc[signal_names]
            self.r2 = self.r ** 2
            
        self.sample_counter = (self.sample_counter - 1) % self.cointegration_period
        
        correlated_slopes = self.r.mul(stddev, axis=1).div(stddev[signal_names], axis=0)
        log_volume = np.log1p(moving_volumes)
        # ideally use mkt cap instead of volume?
        volume_f = pd.DataFrame(log_volume[np.newaxis, :] - log_volume[signal_names][:, np.newaxis], index=signal_names, columns=input_names)
        volume_f = (volume_f * (volume_f > 0) + 1) ** 2
#         print(volume_f)
        
        delta = signals.xs("price",level=1) - moving_prices[signal_names]
        fair_delta_means = correlated_slopes.mul(delta, axis=0)
        delta_vars = self.moving_prices.mse
        correlated_delta_vars = np.square(correlated_slopes).mul(delta_vars[signal_names], axis=0)
        fair_delta_vars = (correlated_delta_vars + (1-self.r2) * self.coint_f * delta_vars).mul(volume_f, axis=0)
        fair_deltas = [Gaussian(fair_delta_means.loc[i], fair_delta_vars.loc[i]) for i in signal_names]
        fair_delta = intersect_with_disagreement(fair_deltas)
        absolute_fair = fair_delta + moving_prices
        
        step = prices - (self.prev_fair.mean + self.moving_prices.trend)
        step_vars = self.moving_err_from_prev_fair.step(step)
        fair_step_means = correlated_slopes.mul(step[signal_names], axis=0)
        correlated_step_vars = np.square(correlated_slopes).mul(step_vars[signal_names], axis=0)
        fair_step_vars = (correlated_step_vars + (1-self.r2) * self.coint_f * step_vars).mul(volume_f, axis=0)
        fair_steps = [Gaussian(fair_step_means.loc[i], fair_step_vars.loc[i]) for i in signal_names]
        fair_step = intersect_with_disagreement(fair_steps)
        relative_fair = fair_step + self.prev_fair + self.moving_prices.trend
        
        fair = intersect_with_disagreement([absolute_fair, relative_fair])
#         print("\n\nASDSA")
#         print(frame["price"])
#         print(absolute_fair)
#         print(relative_fair)
#         print(fair)
#         assert(0)
        self.prev_fair = fair
        return fair[frame.index.unique(0)]

print("Initializing components")

analyze(run(
    KalmanFilter(7500, 90, 3000, 1440, 60, 120, warmup_signals_min, warmup_data_min),
    ExecutionStrategy(1000, 1440, 2, 4, 10, 20, warmup_data_min),
    data_min_.head(2000),
    deepcopy(signal_aggregator_min),
    fees=0.00075,
))

# analyze(run(
#     KalmanFilter(500, 12, 256, 500, 32, 8, warmup_signals_15min, warmup_data_15min),
#     ExecutionStrategy(1000, 500, 1, 3, 3, 10, warmup_data_15min),
#     data_15min_,
#     deepcopy(signal_aggregator_15min),
#     fees=0.00075,
# ))

In [ ]:
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
import pickle

from research.strategy.base import Strategy
from trader.util.stats import Ema, Gaussian
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from trader.util.linalg import orthogonal_projection, hyperplane_projection
from statsmodels.tsa.stattools import adfuller
from scipy.spatial import distance

def johansen(P, maxlag):
    mean = P.mean()
    c = coint_johansen(P / mean - 1, det_order=-1, k_ar_diff=maxlag)
    significant_results = (c.lr1 > c.cvt[:,1]) * (c.lr2 < c.cvm[:,2])
    Q = pd.DataFrame(c.evec[:,significant_results].T, columns=P.columns)
    return mean, Q.div(Q.apply(np.linalg.norm, axis=1), axis=0)

# TODO: reimplement with vector autoregression test, similar to johansen internals.
def test_coint(P, mean_train, q, maxlag):
    x = (P / mean_train - 1) @ q
    return adfuller(x, regression="nc", maxlag=maxlag, autolag=None)[1]

def cosine_similar_to_any(Q, x):
    for q in Q:
        if distance.cosine(q, x) < 0.1:
            return True
    return False

class LiveCointegrator(Strategy):
    def __init__(self, train_size, validation_size, cointegration_period, maxlag):
        self.window_size = train_size + validation_size
        self.train_size = train_size
        self.validation_size = validation_size
        self.cointegration_period = cointegration_period
        self.maxlag = maxlag
        self.sample_counter = 0
        self.price_history = None
        self.Q = None
        self.base_prices = None
#         self.covs = None
        self.base_cov = None
        self.prev_fair = None
        self.coints = []

    def step(self, frame):
        prices = frame["price"]
        
        if self.price_history is None:
            self.price_history = RingBuffer(self.window_size, dtype=(np.float64, len(prices.index)))
            
        if self.prev_fair is None:
            self.prev_fair = self.null_estimate(prices)
            
        self.price_history.append(prices)
        
        if len(self.price_history) < self.window_size:
            return self.null_estimate(prices)
        
        if self.sample_counter == 0:
            P = pd.DataFrame(self.price_history, columns=prices.index)
            P_train = P.iloc[:self.train_size]
            P_val = P.iloc[self.train_size:]
            mean, candidates = johansen(P_train, self.maxlag)
            self.base_prices = mean
            new_coints = [q for q in candidates.values if test_coint(P_val, mean, q, self.maxlag) < .05]
            old_coints = [q for q in self.coints if test_coint(P_val, mean, q, self.maxlag) < .05 and not cosine_similar_to_any(new_coints, q)]
            self.coints = new_coints + old_coints
            self.base_cov = (P / P.mean()).cov()
            
        self.sample_counter -= 1
        self.sample_counter %= self.cointegration_period
        
        if not self.coints:
            return self.null_estimate(prices)
        
        fair_means = [hyperplane_projection(prices / self.base_prices - 1, q) for q in self.coints]
        fair = Gaussian.intersect([(Gaussian(mean, self.base_cov) + 1) * self.base_prices for mean in fair_means])
        
#         step_fair_means = [hyperplane_projection(prices / self.prev_fair.mean - 1, q) for q in self.coints]
#         step_fair = Gaussian.intersect([Gaussian(mean, self.base_cov) * self.prev_fair.mean for mean in step_fair_means])
        
#         fair = fair & (self.prev_fair + step_fair)
        self.prev_fair = fair
        return fair

analyze(run(LiveCointegrator(train_size=500, validation_size = 100, cointegration_period=64, maxlag=8), data_15min, fees=0.001, min_edge=0))

In [ ]:
import numpy as np
import pandas as pd
from numpy_ringbuffer import RingBuffer
import pickle

from research.strategy.base import Strategy
from trader.util.stats import Ema, Gaussian
from trader.util.linalg import orthogonal_projection, hyperplane_projection
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.regression.linear_model import OLS

def plot_gaussian(x, y, gaussian):
    z = gaussian.pdf(np.dstack(np.meshgrid(x,y)))
    plt.pcolormesh(x, y, z)
    plt.show()

class LivePairCointegrator(Strategy):
    def __init__(self, window_size, cointegration_frequency=4):
        self.window_size = window_size
        self.cointegration_period = window_size // cointegration_frequency
        self.sample_counter = 0
        self.price_history = None
        self.prediction_covs = {}

    def step(self, frame):
        prices = frame["price"]
        
        if self.price_history is None:
            self.price_history = RingBuffer(self.window_size, dtype=(np.float64, len(prices.index)))
            
        self.price_history.append(prices)
        
        if len(self.price_history) < self.window_size:
            return self.null_estimate(prices)
        
        df = pd.DataFrame(self.price_history, columns=prices.index)
        self.mean = df.mean()
        deltas = df / self.mean - 1
        stddev = deltas.std() + 1e-100
        regression_slope = deltas.corr().mul(stddev, axis=1).div(stddev, axis=0)
        cov = deltas.cov()
        
        if self.sample_counter == 0:
            
            for pair_i in prices.index:
                for pair_j in prices.index:
                    if pair_i >= pair_j:
                        continue
                    deltas_ij = deltas[[pair_i, pair_j]]
                    regression_vector = [regression_slope.loc[pair_i][pair_j], -1]
                    residuals = orthogonal_projection(deltas_ij, regression_vector)
                    cov_resid = residuals.cov()
                    cov_pred = (deltas_ij - residuals).cov()
                    print(cov_pred)
                    # We can calculate the cointegration p-value more efficiently by reusing
                    # the residuals found via the orthogonal projection but getting it right is
                    # tricky. So we just do it the dumb way
                    p = coint(deltas[pair_j], deltas[pair_i], trend='nc', maxlag=0, autolag=None)[1]
#                     # regularize prediction covariance by using the raw price covariance shape with the 
#                     # prediction covariance slope
#                     cov_sam = deltas_ij.cov()
#                     w_pred, v_pred = np.linalg.eigh(cov_pred)
#                     w_sam, v_sam = np.linalg.eigh(cov_sam)
#                     eigenvalue_ratio_sam = w_sam[1] / w_sam[0]
#                     w_reg = w_pred[1] * np.array([eigenvalue_ratio_sam, 1])
#                     cov_reg = v_pred @ np.diag(w_reg) @ v_pred.T
#                     # discount predictions based on p-value
#                     cov_reg *= max(1, p * p * 900) # min(1e4, np.cosh(16 * p))
#                     cov_reg = pd.DataFrame(cov_reg, index=deltas_ij.columns, columns=deltas_ij.columns)
                    cov_reg = cov_pred + max(1, p * p * 900) * cov_resid
                    self.prediction_covs[pair_i, pair_j] = cov_reg
                    self.prediction_covs[pair_j, pair_i] = cov_reg.loc[::-1, ::-1]
                    
                    # uncomment and run this if you want to see what the prediction covariances look like
                    
                    prediction = Gaussian(pd.Series(0, index=deltas_ij.columns), cov_resid)
                    base = Gaussian(pd.Series(0, index=deltas_ij.columns),  cov_pred)
                    regularized = Gaussian(pd.Series(0, index=deltas_ij.columns), cov_reg)
                    xrange = deltas[pair_i].max() - deltas[pair_i].min()
                    yrange = deltas[pair_j].max() - deltas[pair_j].min()
                    x = np.linspace(-xrange, xrange)
                    y = np.linspace(-yrange, yrange)
                    plot_gaussian(x, y, prediction)
                    plot_gaussian(x, y, base)
                    plot_gaussian(x, y, regularized)
            
        self.sample_counter -= 1
        self.sample_counter %= self.cointegration_period
        
        fairs = []
        delta = prices / self.mean - 1
        cov = deltas.cov()
        for pair_i in prices.index:
            for pair_j in prices.index:
                if pair_i >= pair_j:
                    continue
                regression_vector = [regression_slope.loc[pair_i][pair_j], -1]
#                 residuals = orthogonal_projection(deltas[[pair_i, pair_j]], regression_vector)
                fair_delta_mean = hyperplane_projection(delta[[pair_i, pair_j]], regression_vector)
                fair_cov = self.prediction_covs[pair_i, pair_j]
                fair = Gaussian(fair_delta_mean, fair_cov)
#                 fair = Gaussian(fair_delta_mean, cov.loc[[pair_i, pair_j], [pair_i, pair_j]])
#                 scale = fair.z_score(delta[[pair_i, pair_j]])
#                 fair = Gaussian(fair.mean, fair.covariance * scale)
#                 print(prices)
#                 print(fair)
#                 x = np.linspace(deltas[pair_i].min(), deltas[pair_i].max()) - deltas[pair_i].mean()
#                 y = np.linspace(deltas[pair_j].min(), deltas[pair_j].max()) - deltas[pair_j].mean()
#                 plot_gaussian(x, y, fair)
#                 plot_gaussian(x, y, Gaussian(pd.Series(0, index=[pair_i, pair_j]), residuals.cov()))
#                 plot_gaussian(x, y, Gaussian(pd.Series(0, index=[pair_i, pair_j]), deltas[[pair_i, pair_j]].cov()))
                fairs.append(fair)
#         print(fairs)
        fair = (Gaussian.intersect(fairs) + 1) * self.mean
#         print(prices)
#         print(fair)
#         x = np.linspace(df['BTC_USDT'].min(), df['BTC_USDT'].max())
#         y = np.linspace(df['ETH_USDT'].min(), df['ETH_USDT'].max())
#         plot_gaussian(x, y, fair[['BTC_USDT', 'ETH_USDT']])
#         print(fair / prices - 1)
        return fair

analyze(run(LivePairCointegrator(window_size=96), data_15min.tail(1500), fees=0.001, min_edge=0))

In [ ]:
analyze(run(CointegratorStrategy(cointegration_window_size = 16), data_15min))

In [ ]:
analyze(run(KalmanFilterStrategy(
    correlation_window_size = 480,
    movement_half_life = 1
), tail_data(data_min, 10000), fees = 0.002))

In [ ]:
from strategy import CombinedStrategy

analyze(run(CombinedStrategy([
    KalmanFilterStrategy(correlation_window_size = 60, movement_half_life = 3),
    CointegratorStrategy(cointegration_window_size = 16)
]), data_15min))

In [ ]:
analyze(run(CointegratorStrategy(cointegration_window_size = 512), data_5min))

In [ ]:
analyze(run(KalmanFilterStrategy(correlation_window_size = 165, movement_half_life = 70), data_5min))

In [ ]:
analyze(run(CombinedStrategy([
    KalmanFilterStrategy(correlation_window_size = 16, movement_half_life = 8),
    CointegratorStrategy(cointegration_window_size = 64)
]), tail_data(data_min, 1500)))

In [ ]:
import random

def find_best_window_sizes(data, n):
    points = []
    best = None
    best_ror = 0
    for _ in range(n):
        movement_half_life = random.expovariate(1) * 15
#         window_ratio = random.uniform(1, 10)
#         window_size = max(3, int(movement_half_life * window_ratio))
#         movement_half_life = 2
        window_size = int(random.expovariate(1) * 100) + 400
#         window_size = int(random.expovariate(1) * 30) + 3
#         window_size = 4
#         window_size = 32
        print('Trying window_size: {0} and half_life: {1}'.format(window_size, movement_half_life))
        ror = analyze(run(KalmanFilter(window_size, movement_half_life), data, fees = 0.002), plot=False)
        print('  RoR: {0}'.format(ror))
        point = { 'window_size': window_size, 'half_life': movement_half_life, 'RoR': ror }
        points.append(point)
        if ror > best_ror:
            best = point
            best_ror = ror
    print('Best found:')
    print(best)
    pd.DataFrame(points).plot.scatter('window_size', 'half_life', c='RoR', colormap='jet')
    
find_best_window_sizes(tail_data(data_min, 1500), 25)